In [2]:
!pip install flask
from flask import Flask, request, jsonify
import onnxruntime as ort
import numpy as np

app = Flask(__name__)

# Load the ONNX model
model_path = "../models/LRclassifierLoanPrediction.onnx"
ort_session = ort.InferenceSession(model_path)

# Define the input name
input_name = ort_session.get_inputs()[0].name

@app.route('/predict', methods=['POST'])
def predict():
    # Get the JSON data from the request
    data = request.json
    
    # Convert the JSON data to a NumPy array
    input_data = np.array([
        data["ApplicantIncome"],
        data["CoapplicantIncome"],
        data["LoanAmount"],
        data["Loan_Amount_Term"],
        data["Credit_History"],
        data["Gender"],
        data["Married"],
        data["Dependents_0"],
        data["Dependents_1"],
        data["Dependents_2"],
        data["Dependents_3+"],
        data["Education"],
        data["Self_Employed"],
        data["Property_Area_Rural"],
        data["Property_Area_Semiurban"],
        data["Property_Area_Urban"]
    ], dtype=np.float32).reshape(1, -1)  # Reshape to match the model's expected input shape

    # Run the model
    outputs = ort_session.run(None, {input_name: input_data})
    
    # Retrieve the prediction
    prediction = outputs[0][0]
    
    # Convert the prediction to a human-readable format
    loan_status = "Accepted" if prediction else "Rejected"
    
    # Return the result as a JSON response
    return jsonify({"Loan_Status": loan_status})

if __name__ == '__main__':
    app.run(debug=True, port=5001)  # Change the port number


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/app-root/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/app-root/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 253, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/opt/app-r

SystemExit: 1

/opt/app-root/lib64/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import requests
import json

# Define the URL of the Flask service
url = "http://127.0.0.1:38329/predict"

# Define the JSON payload
payload = {
    "ApplicantIncome": 76.478755,
    "CoapplicantIncome": 0.0,
    "LoanAmount": 12.100089,
    "Loan_Amount_Term": 360.0,
    "Credit_History": 1.0,
    "Gender": True,
    "Married": False,
    "Dependents_0": True,
    "Dependents_1": False,
    "Dependents_2": False,
    "Dependents_3+": False,
    "Education": True,
    "Self_Employed": False,
    "Property_Area_Rural": False,
    "Property_Area_Semiurban": False,
    "Property_Area_Urban": True
}

# Send the POST request
response = requests.post(url, json=payload)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())